In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split, GridSearchCV, KFold, StratifiedKFold, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn import metrics
import openpyxl
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import Pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import RocCurveDisplay
from tqdm.auto import tqdm
import time
from scipy.stats import wilcoxon

# STEP 1

In [ ]:
# Load data
file_path = '/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/osr_rdx_suv_aftercombat_ScannerHarmonization.csv'
df_step1 = pd.read_csv(file_path)

# Prepare features and target
X1 = df_step1[['"original_glszm_ZoneEntropy"', '"original_shape_LeastAxisLength"']].values
y1 = df_step1['outcome'].values

# Initialize models and preprocessing steps
scaler = StandardScaler()
oversampler = SMOTE(sampling_strategy='auto', random_state=42, k_neighbors=3)
randomforest = RandomForestClassifier()
svm = SVC(probability=True)
lr = LogisticRegression()

# Create pipelines
pipeline_rf = Pipeline(steps=[("oversampler", oversampler), ("model_rf", randomforest)])
pipeline_svm = Pipeline(steps=[("oversampler", oversampler), ("model_svm", svm)])
pipeline_lr = Pipeline(steps=[("oversampler", oversampler), ("model_lr", lr)])

# Define hyperparameter grids
rf_params = {'model_rf__n_estimators': [25, 50, 75, 100, 125, 150],
             'model_rf__min_samples_split': [2, 3],
             'model_rf__min_samples_leaf': [1, 2],
             'model_rf__max_features': [2]}

svm_params = {'model_svm__C': [0.5, 1, 1.5, 2, 2.5, 3],
              'model_svm__gamma': [0.5, 0.8, 1, 1.5],
              'model_svm__kernel': ['linear'],
              'model_svm__probability': [True]}

lr_params = {'model_lr__solver': ['lbfgs', 'newton-cg', 'liblinear'],
             'model_lr__penalty': ['l2'],
             'model_lr__C': [0.1, 0.5, 0.8, 1, 1.25, 2]}

# Initialize lists to store metrics
num_zeros_list, num_ones_list = [], []
rf_train_metrics_list, svm_train_metrics_list, lr_train_metrics_list = [], [], []
rf_metrics_list, svm_metrics_list, lr_metrics_list, majority_metrics_list = [], [], [], []
rf_all_fpr, rf_all_tpr, svm_all_fpr, svm_all_tpr, lr_all_fpr, lr_all_tpr, majority_all_fpr, majority_all_tpr = [], [], [], [], [], [], [], []

# Loop over iterations
for i in tqdm(range(100)):
    time.sleep(0.1)
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, stratify=y1, random_state=i)
    
    # Scale features
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Resample training data
    X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train_scaled, y_train)

    # Grid search for each model
    rf_gs = GridSearchCV(pipeline_rf, rf_params, scoring='roc_auc')
    svm_gs = GridSearchCV(pipeline_svm, svm_params, scoring='roc_auc')
    lr_gs = GridSearchCV(pipeline_lr, lr_params, scoring='roc_auc')

    rf_gs.fit(X_train_scaled, y_train)
    svm_gs.fit(X_train_scaled, y_train)
    lr_gs.fit(X_train_scaled, y_train)

    # Training AUC and hyperparameters
    rf_train_auc = rf_gs.best_score_
    rf_train_metrics_list.append({'AUC': rf_train_auc, 'Hyperparameters': rf_gs.best_params_})

    svm_train_auc = svm_gs.best_score_
    svm_train_metrics_list.append({'AUC': svm_train_auc, 'Hyperparameters': svm_gs.best_params_})

    lr_train_auc = lr_gs.best_score_
    lr_train_metrics_list.append({'AUC': lr_train_auc, 'Hyperparameters': lr_gs.best_params_})

    # Predictions and probabilities
    rf_pred = rf_gs.predict(X_test_scaled)
    svm_pred = svm_gs.predict(X_test_scaled)
    lr_pred = lr_gs.predict(X_test_scaled)

    rf_proba = rf_gs.predict_proba(X_test_scaled)[:, 1]
    svm_proba = svm_gs.predict_proba(X_test_scaled)[:, 1]
    lr_proba = lr_gs.predict_proba(X_test_scaled)[:, 1]

    # Test set metrics for each model
    rf_auc = roc_auc_score(y_test, rf_proba)
    svm_auc = roc_auc_score(y_test, svm_proba)
    lr_auc = roc_auc_score(y_test, lr_proba)

    rf_metrics_list.append({'AUC': rf_auc, 'Accuracy': accuracy_score(y_test, rf_pred), 'Sensitivity': confusion_matrix(y_test, rf_pred)[1, 1] / (confusion_matrix(y_test, rf_pred)[1, 1] + confusion_matrix(y_test, rf_pred)[1, 0]),
                            'Specificity': confusion_matrix(y_test, rf_pred)[0, 0] / (confusion_matrix(y_test, rf_pred)[0, 0] + confusion_matrix(y_test, rf_pred)[0, 1]),
                            'PPV': confusion_matrix(y_test, rf_pred)[1, 1] / (confusion_matrix(y_test, rf_pred)[1, 1] + confusion_matrix(y_test, rf_pred)[0, 1]),
                            'NPV': confusion_matrix(y_test, rf_pred)[0, 0] / (confusion_matrix(y_test, rf_pred)[0, 0] + confusion_matrix(y_test, rf_pred)[1, 0]),
                            'Hyperparameters': rf_gs.best_params_})

    svm_metrics_list.append({'AUC': svm_auc, 'Accuracy': accuracy_score(y_test, svm_pred), 'Sensitivity': confusion_matrix(y_test, svm_pred)[1, 1] / (confusion_matrix(y_test, svm_pred)[1, 1] + confusion_matrix(y_test, svm_pred)[1, 0]),
                             'Specificity': confusion_matrix(y_test, svm_pred)[0, 0] / (confusion_matrix(y_test, svm_pred)[0, 0] + confusion_matrix(y_test, svm_pred)[0, 1]),
                             'PPV': confusion_matrix(y_test, svm_pred)[1, 1] / (confusion_matrix(y_test, svm_pred)[1, 1] + confusion_matrix(y_test, svm_pred)[0, 1]),
                             'NPV': confusion_matrix(y_test, svm_pred)[0, 0] / (confusion_matrix(y_test, svm_pred)[0, 0] + confusion_matrix(y_test, svm_pred)[1, 0]),
                             'Hyperparameters': svm_gs.best_params_})

    lr_metrics_list.append({'AUC': lr_auc, 'Accuracy': accuracy_score(y_test, lr_pred), 'Sensitivity': confusion_matrix(y_test, lr_pred)[1, 1] / (confusion_matrix(y_test, lr_pred)[1, 1] + confusion_matrix(y_test, lr_pred)[1, 0]),
                            'Specificity': confusion_matrix(y_test, lr_pred)[0, 0] / (confusion_matrix(y_test, lr_pred)[0, 0] + confusion_matrix(y_test, lr_pred)[0, 1]),
                            'PPV': confusion_matrix(y_test, lr_pred)[1, 1] / (confusion_matrix(y_test, lr_pred)[1, 1] + confusion_matrix(y_test, lr_pred)[0, 1]),
                            'NPV': confusion_matrix(y_test, lr_pred)[0, 0] / (confusion_matrix(y_test, lr_pred)[0, 0] + confusion_matrix(y_test, lr_pred)[1, 0]),
                            'Hyperparameters': lr_gs.best_params_})

    # ROC curve for each model
    rf_fpr, rf_tpr, _ = roc_curve(y_test, rf_proba, drop_intermediate=False)
    interp_rf_tpr = np.interp(np.linspace(0, 1, 100), rf_fpr, rf_tpr)
    rf_all_fpr.append(np.linspace(0, 1, 100))
    rf_all_tpr.append(interp_rf_tpr)

    svm_fpr, svm_tpr, _ = roc_curve(y_test, svm_proba, drop_intermediate=False)
    interp_svm_tpr = np.interp(np.linspace(0, 1, 100), svm_fpr, svm_tpr)
    svm_all_fpr.append(np.linspace(0, 1, 100))
    svm_all_tpr.append(interp_svm_tpr)

    lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_proba, drop_intermediate=False)
    interp_lr_tpr = np.interp(np.linspace(0, 1, 100), lr_fpr, lr_tpr)
    lr_all_fpr.append(np.linspace(0, 1, 100))
    lr_all_tpr.append(interp_lr_tpr)

    # Majority Vote model
    clf = VotingClassifier(estimators=[
    ('rf', RandomForestClassifier(**{key.split('__')[1]: value for key, value in rf_gs.best_params_.items()})),
    ('svm', SVC(**{key.split('__')[1]: value for key, value in svm_gs.best_params_.items()})),
    ('lr', LogisticRegression(**{key.split('__')[1]: value for key, value in lr_gs.best_params_.items()}))
    ], voting='soft', flatten_transform=True)

    clf.fit(X_train_resampled, y_train_resampled)
    y_pred = clf.predict(X_test_scaled)
    y_proba = clf.predict_proba(X_test_scaled)[:, 1]

    # Majority Vote metrics
    majority_auc = roc_auc_score(y_test, y_proba)
    majority_cm = confusion_matrix(y_test, y_pred)
    majority_accuracy = accuracy_score(y_test, y_pred)

    majority_sensitivity = majority_cm[1, 1] / (majority_cm[1, 1] + majority_cm[1, 0])
    majority_specificity = majority_cm[0, 0] / (majority_cm[0, 0] + majority_cm[0, 1])
    majority_ppv = majority_cm[1, 1] / (majority_cm[1, 1] + majority_cm[0, 1])
    majority_npv = majority_cm[0, 0] / (majority_cm[0, 0] + majority_cm[1, 0])

    majority_metrics_list.append({'AUC': majority_auc, 'Accuracy': majority_accuracy,
                                  'Sensitivity': majority_sensitivity, 'Specificity': majority_specificity,
                                  'PPV': majority_ppv, 'NPV': majority_npv})

    majority_fpr, majority_tpr, _ = roc_curve(y_test, y_proba, drop_intermediate=False)
    interp_majority_tpr = np.interp(np.linspace(0, 1, 100), majority_fpr, majority_tpr)
    majority_all_fpr.append(np.linspace(0, 1, 100))
    majority_all_tpr.append(interp_majority_tpr)

    # Store class distribution in each iteration
    num_zeros_list.append(np.sum(y_test == 0))
    num_ones_list.append(np.sum(y_test == 1))

# Create DataFrames for metrics
rf_train_metrics_df = pd.DataFrame(rf_train_metrics_list)
svm_train_metrics_df = pd.DataFrame(svm_train_metrics_list)
lr_train_metrics_df = pd.DataFrame(lr_train_metrics_list)

rf_metrics_df_soft = pd.DataFrame(rf_metrics_list)
svm_metrics_df_soft = pd.DataFrame(svm_metrics_list)
lr_metrics_df_soft = pd.DataFrame(lr_metrics_list)
majority_metrics_df_soft = pd.DataFrame(majority_metrics_list)

# Plot ROC curves
plt.figure(figsize=(8, 6))

rf_mean_fpr = np.mean(rf_all_fpr, axis=0)
rf_mean_tpr = np.mean(rf_all_tpr, axis=0)
plt.plot(rf_mean_fpr, rf_mean_tpr, label=f'Random Forest AUC = {np.mean(rf_metrics_df_soft["AUC"]):.3f}')


svm_mean_fpr = np.mean(svm_all_fpr, axis=0)
svm_mean_tpr = np.mean(svm_all_tpr, axis=0)
plt.plot(svm_mean_fpr, svm_mean_tpr, label=f'SVM AUC = {np.mean(svm_metrics_df_soft["AUC"]):.3f}')

lr_mean_fpr = np.mean(lr_all_fpr, axis=0)
lr_mean_tpr = np.mean(lr_all_tpr, axis=0)
lr_mean_fpr = np.concatenate([[0], lr_mean_fpr])
lr_mean_tpr = np.concatenate([[0], lr_mean_tpr])
plt.plot(lr_mean_fpr, lr_mean_tpr, label=f'Logistic Regression AUC = {np.mean(lr_metrics_df_soft["AUC"]):.3f}')

majority_mean_fpr = np.mean(majority_all_fpr, axis=0)
majority_mean_tpr = np.mean(majority_all_tpr, axis=0)
majority_mean_fpr = np.concatenate([[0], majority_mean_fpr])
majority_mean_tpr = np.concatenate([[0], majority_mean_tpr])
plt.plot(majority_mean_fpr, majority_mean_tpr, label=f'Majority Vote AUC = {np.mean(majority_metrics_df_soft["AUC"]):.3f}')

plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('')
plt.legend()
plt.show()

# Create a DataFrame for class distribution
sanity_check_df = pd.DataFrame({'Num_Zeros': num_zeros_list, 'Num_Ones': num_ones_list})

# Print average AUC values for each model in both training and test sets
print(f'The average AUC for Random Forest in the training set is: {rf_train_metrics_df["AUC"].mean()}')
print(f'The average AUC for SVM in the training set is: {svm_train_metrics_df["AUC"].mean()}')
print(f'The average AUC for Logistic Regression in the training set is: {lr_train_metrics_df["AUC"].mean()}')
print()
print(f'The average AUC for Random Forest in the test set is: {rf_metrics_df_soft["AUC"].mean()}')
print(f'The average AUC for SVM in the test set is: {svm_metrics_df_soft["AUC"].mean()}')
print(f'The average AUC for Logistic Regression in the test set is: {lr_metrics_df_soft["AUC"].mean()}')
print(f'The average AUC for the Majority Vote model in the test set is: {majority_metrics_df_soft["AUC"].mean()}')

In [ ]:
rf_train_metrics_df.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP1_train_results_rf.xlsx')
svm_train_metrics_df.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP1_train_results_svm.xlsx')
lr_train_metrics_df.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP1_train_results_lr.xlsx')

rf_metrics_df_soft.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP1_results_rf.xlsx')
svm_metrics_df_soft.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP1_results_svm.xlsx')
lr_metrics_df_soft.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP1_results_lr.xlsx')
majority_metrics_df_soft.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP1_results_majority.xlsx')

# STEP 2

In [ ]:
# Load data
file_path = '/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/stn_rdx_rp.xlsx'
df_step2 = pd.read_excel(file_path)
file_path_outcome = '/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_radiomics/DB_STN_PET_RP.xlsx'
df_step2_outcome = pd.read_excel(file_path_outcome)
df_step2complete = pd.merge(df_step2, df_step2_outcome, on='ID')

# Prepare features and target
X2 = df_step2complete[['original_glszm_ZoneEntropy', 'original_shape_LeastAxisLength']].values
y2 = df_step2complete['ISUP_binary'].values

# Initialize models and preprocessing steps
scaler = StandardScaler()
oversampler = SMOTE(sampling_strategy='auto', random_state=42, k_neighbors=3)
randomforest = RandomForestClassifier()
svm = SVC(probability=True)
lr = LogisticRegression()

# Create pipelines
pipeline_rf = Pipeline(steps=[("oversampler", oversampler), ("model_rf", randomforest)])
pipeline_svm = Pipeline(steps=[("oversampler", oversampler), ("model_svm", svm)])
pipeline_lr = Pipeline(steps=[("oversampler", oversampler), ("model_lr", lr)])

# Define hyperparameter grids
rf_params = {'model_rf__n_estimators': [25, 50, 75, 100, 125, 150],
             'model_rf__min_samples_split': [2, 3],
             'model_rf__min_samples_leaf': [1, 2],
             'model_rf__max_features': [2]}

svm_params = {'model_svm__C': [0.5, 1, 1.5, 2, 2.5, 3],
              'model_svm__gamma': [0.5, 0.8, 1, 1.5],
              'model_svm__kernel': ['linear'],
              'model_svm__probability': [True]}

lr_params = {'model_lr__solver': ['lbfgs', 'newton-cg', 'liblinear'],
             'model_lr__penalty': ['l2'],
             'model_lr__C': [0.1, 0.5, 0.8, 1, 1.25, 2]}

# Initialize lists to store metrics
num_zeros_list, num_ones_list = [], []
rf_train_metrics_list, svm_train_metrics_list, lr_train_metrics_list = [], [], []
rf_metrics_list, svm_metrics_list, lr_metrics_list, majority_metrics_list = [], [], [], []
rf_all_fpr, rf_all_tpr, svm_all_fpr, svm_all_tpr, lr_all_fpr, lr_all_tpr, majority_all_fpr, majority_all_tpr = [], [], [], [], [], [], [], []

# Loop over iterations
for i in tqdm(range(100)):
    time.sleep(0.1)
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.3, stratify=y2, random_state=i)
    
    # Scale features
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Resample training data
    X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train_scaled, y_train)

    # Grid search for each model
    rf_gs = GridSearchCV(pipeline_rf, rf_params, scoring='roc_auc')
    svm_gs = GridSearchCV(pipeline_svm, svm_params, scoring='roc_auc')
    lr_gs = GridSearchCV(pipeline_lr, lr_params, scoring='roc_auc')

    rf_gs.fit(X_train_scaled, y_train)
    svm_gs.fit(X_train_scaled, y_train)
    lr_gs.fit(X_train_scaled, y_train)

    # Training AUC and hyperparameters
    rf_train_auc = rf_gs.best_score_
    rf_train_metrics_list.append({'AUC': rf_train_auc, 'Hyperparameters': rf_gs.best_params_})

    svm_train_auc = svm_gs.best_score_
    svm_train_metrics_list.append({'AUC': svm_train_auc, 'Hyperparameters': svm_gs.best_params_})

    lr_train_auc = lr_gs.best_score_
    lr_train_metrics_list.append({'AUC': lr_train_auc, 'Hyperparameters': lr_gs.best_params_})

    # Predictions and probabilities
    rf_pred = rf_gs.predict(X_test_scaled)
    svm_pred = svm_gs.predict(X_test_scaled)
    lr_pred = lr_gs.predict(X_test_scaled)

    rf_proba = rf_gs.predict_proba(X_test_scaled)[:, 1]
    svm_proba = svm_gs.predict_proba(X_test_scaled)[:, 1]
    lr_proba = lr_gs.predict_proba(X_test_scaled)[:, 1]

    # Test set metrics for each model
    rf_auc = roc_auc_score(y_test, rf_proba)
    svm_auc = roc_auc_score(y_test, svm_proba)
    lr_auc = roc_auc_score(y_test, lr_proba)

    rf_metrics_list.append({'AUC': rf_auc, 'Accuracy': accuracy_score(y_test, rf_pred), 'Sensitivity': confusion_matrix(y_test, rf_pred)[1, 1] / (confusion_matrix(y_test, rf_pred)[1, 1] + confusion_matrix(y_test, rf_pred)[1, 0]),
                            'Specificity': confusion_matrix(y_test, rf_pred)[0, 0] / (confusion_matrix(y_test, rf_pred)[0, 0] + confusion_matrix(y_test, rf_pred)[0, 1]),
                            'PPV': confusion_matrix(y_test, rf_pred)[1, 1] / (confusion_matrix(y_test, rf_pred)[1, 1] + confusion_matrix(y_test, rf_pred)[0, 1]),
                            'NPV': confusion_matrix(y_test, rf_pred)[0, 0] / (confusion_matrix(y_test, rf_pred)[0, 0] + confusion_matrix(y_test, rf_pred)[1, 0]),
                            'Hyperparameters': rf_gs.best_params_})

    svm_metrics_list.append({'AUC': svm_auc, 'Accuracy': accuracy_score(y_test, svm_pred), 'Sensitivity': confusion_matrix(y_test, svm_pred)[1, 1] / (confusion_matrix(y_test, svm_pred)[1, 1] + confusion_matrix(y_test, svm_pred)[1, 0]),
                             'Specificity': confusion_matrix(y_test, svm_pred)[0, 0] / (confusion_matrix(y_test, svm_pred)[0, 0] + confusion_matrix(y_test, svm_pred)[0, 1]),
                             'PPV': confusion_matrix(y_test, svm_pred)[1, 1] / (confusion_matrix(y_test, svm_pred)[1, 1] + confusion_matrix(y_test, svm_pred)[0, 1]),
                             'NPV': confusion_matrix(y_test, svm_pred)[0, 0] / (confusion_matrix(y_test, svm_pred)[0, 0] + confusion_matrix(y_test, svm_pred)[1, 0]),
                             'Hyperparameters': svm_gs.best_params_})

    lr_metrics_list.append({'AUC': lr_auc, 'Accuracy': accuracy_score(y_test, lr_pred), 'Sensitivity': confusion_matrix(y_test, lr_pred)[1, 1] / (confusion_matrix(y_test, lr_pred)[1, 1] + confusion_matrix(y_test, lr_pred)[1, 0]),
                            'Specificity': confusion_matrix(y_test, lr_pred)[0, 0] / (confusion_matrix(y_test, lr_pred)[0, 0] + confusion_matrix(y_test, lr_pred)[0, 1]),
                            'PPV': confusion_matrix(y_test, lr_pred)[1, 1] / (confusion_matrix(y_test, lr_pred)[1, 1] + confusion_matrix(y_test, lr_pred)[0, 1]),
                            'NPV': confusion_matrix(y_test, lr_pred)[0, 0] / (confusion_matrix(y_test, lr_pred)[0, 0] + confusion_matrix(y_test, lr_pred)[1, 0]),
                            'Hyperparameters': lr_gs.best_params_})

    # ROC curve for each model
    rf_fpr, rf_tpr, _ = roc_curve(y_test, rf_proba, drop_intermediate=False)
    interp_rf_tpr = np.interp(np.linspace(0, 1, 100), rf_fpr, rf_tpr)
    rf_all_fpr.append(np.linspace(0, 1, 100))
    rf_all_tpr.append(interp_rf_tpr)

    svm_fpr, svm_tpr, _ = roc_curve(y_test, svm_proba, drop_intermediate=False)
    interp_svm_tpr = np.interp(np.linspace(0, 1, 100), svm_fpr, svm_tpr)
    svm_all_fpr.append(np.linspace(0, 1, 100))
    svm_all_tpr.append(interp_svm_tpr)

    lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_proba, drop_intermediate=False)
    interp_lr_tpr = np.interp(np.linspace(0, 1, 100), lr_fpr, lr_tpr)
    lr_all_fpr.append(np.linspace(0, 1, 100))
    lr_all_tpr.append(interp_lr_tpr)

    # Majority Vote model
    clf = VotingClassifier(estimators=[
    ('rf', RandomForestClassifier(**{key.split('__')[1]: value for key, value in rf_gs.best_params_.items()})),
    ('svm', SVC(**{key.split('__')[1]: value for key, value in svm_gs.best_params_.items()})),
    ('lr', LogisticRegression(**{key.split('__')[1]: value for key, value in lr_gs.best_params_.items()}))
    ], voting='soft', flatten_transform=True)


    clf.fit(X_train_resampled, y_train_resampled)
    y_pred = clf.predict(X_test_scaled)
    y_proba = clf.predict_proba(X_test_scaled)[:, 1]

    # Majority Vote metrics
    majority_auc = roc_auc_score(y_test, y_proba)
    majority_cm = confusion_matrix(y_test, y_pred)
    majority_accuracy = accuracy_score(y_test, y_pred)

    majority_sensitivity = majority_cm[1, 1] / (majority_cm[1, 1] + majority_cm[1, 0])
    majority_specificity = majority_cm[0, 0] / (majority_cm[0, 0] + majority_cm[0, 1])
    majority_ppv = majority_cm[1, 1] / (majority_cm[1, 1] + majority_cm[0, 1])
    majority_npv = majority_cm[0, 0] / (majority_cm[0, 0] + majority_cm[1, 0])

    majority_metrics_list.append({'AUC': majority_auc, 'Accuracy': majority_accuracy,
                                  'Sensitivity': majority_sensitivity, 'Specificity': majority_specificity,
                                  'PPV': majority_ppv, 'NPV': majority_npv})

    majority_fpr, majority_tpr, _ = roc_curve(y_test, y_proba, drop_intermediate=False)
    interp_majority_tpr = np.interp(np.linspace(0, 1, 100), majority_fpr, majority_tpr)
    majority_all_fpr.append(np.linspace(0, 1, 100))
    majority_all_tpr.append(interp_majority_tpr)

    # Store class distribution in each iteration
    num_zeros_list.append(np.sum(y_test == 0))
    num_ones_list.append(np.sum(y_test == 1))

# Create DataFrames for metrics
rf_train_metrics_df = pd.DataFrame(rf_train_metrics_list)
svm_train_metrics_df = pd.DataFrame(svm_train_metrics_list)
lr_train_metrics_df = pd.DataFrame(lr_train_metrics_list)

rf_metrics_df_soft = pd.DataFrame(rf_metrics_list)
svm_metrics_df_soft = pd.DataFrame(svm_metrics_list)
lr_metrics_df_soft = pd.DataFrame(lr_metrics_list)
majority_metrics_df_soft = pd.DataFrame(majority_metrics_list)

# Plot ROC curves
plt.figure(figsize=(8, 6))

rf_mean_fpr = np.mean(rf_all_fpr, axis=0)
rf_mean_tpr = np.mean(rf_all_tpr, axis=0)
plt.plot(rf_mean_fpr, rf_mean_tpr, label=f'Random Forest AUC = {np.mean(rf_metrics_df_soft["AUC"]):.3f}')


svm_mean_fpr = np.mean(svm_all_fpr, axis=0)
svm_mean_tpr = np.mean(svm_all_tpr, axis=0)
plt.plot(svm_mean_fpr, svm_mean_tpr, label=f'SVM AUC = {np.mean(svm_metrics_df_soft["AUC"]):.3f}')

lr_mean_fpr = np.mean(lr_all_fpr, axis=0)
lr_mean_tpr = np.mean(lr_all_tpr, axis=0)
lr_mean_fpr = np.concatenate([[0], lr_mean_fpr])
lr_mean_tpr = np.concatenate([[0], lr_mean_tpr])
plt.plot(lr_mean_fpr, lr_mean_tpr, label=f'Logistic Regression AUC = {np.mean(lr_metrics_df_soft["AUC"]):.3f}')

majority_mean_fpr = np.mean(majority_all_fpr, axis=0)
majority_mean_tpr = np.mean(majority_all_tpr, axis=0)
majority_mean_fpr = np.concatenate([[0], majority_mean_fpr])
majority_mean_tpr = np.concatenate([[0], majority_mean_tpr])
plt.plot(majority_mean_fpr, majority_mean_tpr, label=f'Majority Vote AUC = {np.mean(majority_metrics_df_soft["AUC"]):.3f}')

plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('')
plt.legend()
plt.show()

# Create a DataFrame for class distribution
sanity_check_df = pd.DataFrame({'Num_Zeros': num_zeros_list, 'Num_Ones': num_ones_list})

# Print average AUC values for each model in both training and test sets
print(f'The average AUC for Random Forest in the training set is: {rf_train_metrics_df["AUC"].mean()}')
print(f'The average AUC for SVM in the training set is: {svm_train_metrics_df["AUC"].mean()}')
print(f'The average AUC for Logistic Regression in the training set is: {lr_train_metrics_df["AUC"].mean()}')
print()
print(f'The average AUC for Random Forest in the test set is: {rf_metrics_df_soft["AUC"].mean()}')
print(f'The average AUC for SVM in the test set is: {svm_metrics_df_soft["AUC"].mean()}')
print(f'The average AUC for Logistic Regression in the test set is: {lr_metrics_df_soft["AUC"].mean()}')
print(f'The average AUC for the Majority Vote model in the test set is: {majority_metrics_df_soft["AUC"].mean()}')


In [ ]:
rf_train_metrics_df.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP2_train_results_rf.xlsx')
svm_train_metrics_df.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP2_train_results_svm.xlsx')
lr_train_metrics_df.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP2_train_results_lr.xlsx')

rf_metrics_df_soft.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP2_results_rf.xlsx')
svm_metrics_df_soft.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP2_results_svm.xlsx')
lr_metrics_df_soft.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP2_results_lr.xlsx')
majority_metrics_df_soft.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP2_results_majority.xlsx')

# STEP 3

In [ ]:
# Define DeLong test

def delong_test(y_true, y_score1, y_score2):
    n = len(y_true)
    auc1 = roc_auc_score(y_true, y_score1)
    auc2 = roc_auc_score(y_true, y_score2)
    u_stat, p_value = wilcoxon(x=y_score1, y=y_score2)
    return u_stat, p_value

In [ ]:
# Load data
file_path = '/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/df_step3_4_5.csv'
df_step3 = pd.read_csv(file_path)
df_train_step3 = df_step3.iloc[0:62,]
df_test_step3 =df_step3.iloc[62:,]

X_train = df_train_step3[['"original_glszm_ZoneEntropy"', '"original_shape_LeastAxisLength"']].values
y_train = df_train_step3['x'].values
X_test = df_test_step3[['"original_glszm_ZoneEntropy"', '"original_shape_LeastAxisLength"']].values
y_test = df_test_step3['x'].values

# Initialize models and preprocessing steps
scaler = StandardScaler()
oversampler = SMOTE(sampling_strategy='auto', random_state=42, k_neighbors=3)
randomforest = RandomForestClassifier()
svm = SVC(probability=True)
lr = LogisticRegression()

# Create pipelines
pipeline_rf = Pipeline(steps=[("oversampler", oversampler), ("model_rf", randomforest)])
pipeline_svm = Pipeline(steps=[("oversampler", oversampler), ("model_svm", svm)])
pipeline_lr = Pipeline(steps=[("oversampler", oversampler), ("model_lr", lr)])

# Define hyperparameter grids
rf_params = {'model_rf__n_estimators': [25, 50, 75, 100, 125, 150],
             'model_rf__min_samples_split': [2, 3],
             'model_rf__min_samples_leaf': [1, 2],
             'model_rf__max_features': [2]}

svm_params = {'model_svm__C': [0.5, 1, 1.5, 2, 2.5, 3],
              'model_svm__gamma': [0.5, 0.8, 1, 1.5],
              'model_svm__kernel': ['linear'],
              'model_svm__probability': [True]}

lr_params = {'model_lr__solver': ['lbfgs', 'newton-cg', 'liblinear'],
             'model_lr__penalty': ['l2'],
             'model_lr__C': [0.1, 0.5, 0.8, 1, 1.25, 2]}

# Initialize lists to store metrics
rf_train_metrics_list, svm_train_metrics_list, lr_train_metrics_list = [], [], []
rf_metrics_list, svm_metrics_list, lr_metrics_list, majority_metrics_list = [], [], [], []
    
# Scale features
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
    
# Resample training data
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train_scaled, y_train)

# Grid search for each model
rf_gs = GridSearchCV(pipeline_rf, rf_params, scoring='roc_auc')
svm_gs = GridSearchCV(pipeline_svm, svm_params, scoring='roc_auc')
lr_gs = GridSearchCV(pipeline_lr, lr_params, scoring='roc_auc')

rf_gs.fit(X_train_scaled, y_train)
svm_gs.fit(X_train_scaled, y_train)
lr_gs.fit(X_train_scaled, y_train)

# Training AUC and hyperparameters
rf_train_auc = rf_gs.best_score_
rf_train_metrics_list.append({'AUC': rf_train_auc, 'Hyperparameters': rf_gs.best_params_})

svm_train_auc = svm_gs.best_score_
svm_train_metrics_list.append({'AUC': svm_train_auc, 'Hyperparameters': svm_gs.best_params_})

lr_train_auc = lr_gs.best_score_
lr_train_metrics_list.append({'AUC': lr_train_auc, 'Hyperparameters': lr_gs.best_params_})

# Predictions and probabilities
rf_pred = rf_gs.predict(X_test_scaled)
svm_pred = svm_gs.predict(X_test_scaled)
lr_pred = lr_gs.predict(X_test_scaled)

rf_proba = rf_gs.predict_proba(X_test_scaled)[:, 1]
svm_proba = svm_gs.predict_proba(X_test_scaled)[:, 1]
lr_proba = lr_gs.predict_proba(X_test_scaled)[:, 1]

# Test set metrics for each model
rf_auc = roc_auc_score(y_test, rf_proba)
svm_auc = roc_auc_score(y_test, svm_proba)
lr_auc = roc_auc_score(y_test, lr_proba)

rf_metrics_list.append({'AUC': rf_auc, 'Accuracy': accuracy_score(y_test, rf_pred), 'Sensitivity': confusion_matrix(y_test, rf_pred)[1, 1] / (confusion_matrix(y_test, rf_pred)[1, 1] + confusion_matrix(y_test, rf_pred)[1, 0]),
                            'Specificity': confusion_matrix(y_test, rf_pred)[0, 0] / (confusion_matrix(y_test, rf_pred)[0, 0] + confusion_matrix(y_test, rf_pred)[0, 1]),
                            'PPV': confusion_matrix(y_test, rf_pred)[1, 1] / (confusion_matrix(y_test, rf_pred)[1, 1] + confusion_matrix(y_test, rf_pred)[0, 1]),
                            'NPV': confusion_matrix(y_test, rf_pred)[0, 0] / (confusion_matrix(y_test, rf_pred)[0, 0] + confusion_matrix(y_test, rf_pred)[1, 0]),
                            'Hyperparameters': rf_gs.best_params_})

svm_metrics_list.append({'AUC': svm_auc, 'Accuracy': accuracy_score(y_test, svm_pred), 'Sensitivity': confusion_matrix(y_test, svm_pred)[1, 1] / (confusion_matrix(y_test, svm_pred)[1, 1] + confusion_matrix(y_test, svm_pred)[1, 0]),
                             'Specificity': confusion_matrix(y_test, svm_pred)[0, 0] / (confusion_matrix(y_test, svm_pred)[0, 0] + confusion_matrix(y_test, svm_pred)[0, 1]),
                             'PPV': confusion_matrix(y_test, svm_pred)[1, 1] / (confusion_matrix(y_test, svm_pred)[1, 1] + confusion_matrix(y_test, svm_pred)[0, 1]),
                             'NPV': confusion_matrix(y_test, svm_pred)[0, 0] / (confusion_matrix(y_test, svm_pred)[0, 0] + confusion_matrix(y_test, svm_pred)[1, 0]),
                             'Hyperparameters': svm_gs.best_params_})

lr_metrics_list.append({'AUC': lr_auc, 'Accuracy': accuracy_score(y_test, lr_pred), 'Sensitivity': confusion_matrix(y_test, lr_pred)[1, 1] / (confusion_matrix(y_test, lr_pred)[1, 1] + confusion_matrix(y_test, lr_pred)[1, 0]),
                            'Specificity': confusion_matrix(y_test, lr_pred)[0, 0] / (confusion_matrix(y_test, lr_pred)[0, 0] + confusion_matrix(y_test, lr_pred)[0, 1]),
                            'PPV': confusion_matrix(y_test, lr_pred)[1, 1] / (confusion_matrix(y_test, lr_pred)[1, 1] + confusion_matrix(y_test, lr_pred)[0, 1]),
                            'NPV': confusion_matrix(y_test, lr_pred)[0, 0] / (confusion_matrix(y_test, lr_pred)[0, 0] + confusion_matrix(y_test, lr_pred)[1, 0]),
                            'Hyperparameters': lr_gs.best_params_})

# ROC curve for each model
rf_fpr, rf_tpr, _ = roc_curve(y_test, rf_proba, drop_intermediate=False)
svm_fpr, svm_tpr, _ = roc_curve(y_test, svm_proba, drop_intermediate=False)
lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_proba, drop_intermediate=False)


# Majority Vote model
clf = VotingClassifier(estimators=[
    ('rf', RandomForestClassifier(**{key.split('__')[1]: value for key, value in rf_gs.best_params_.items()})),
    ('svm', SVC(**{key.split('__')[1]: value for key, value in svm_gs.best_params_.items()})),
    ('lr', LogisticRegression(**{key.split('__')[1]: value for key, value in lr_gs.best_params_.items()}))
], voting='soft', flatten_transform=True)

clf.fit(X_train_resampled, y_train_resampled)
y_pred = clf.predict(X_test_scaled)
y_proba = clf.predict_proba(X_test_scaled)[:, 1]

# Majority Vote metrics
majority_auc = roc_auc_score(y_test, y_proba)
majority_cm = confusion_matrix(y_test, y_pred)
majority_accuracy = accuracy_score(y_test, y_pred)

majority_sensitivity = majority_cm[1, 1] / (majority_cm[1, 1] + majority_cm[1, 0])
majority_specificity = majority_cm[0, 0] / (majority_cm[0, 0] + majority_cm[0, 1])
majority_ppv = majority_cm[1, 1] / (majority_cm[1, 1] + majority_cm[0, 1])
majority_npv = majority_cm[0, 0] / (majority_cm[0, 0] + majority_cm[1, 0])

majority_metrics_list.append({'AUC': majority_auc, 'Accuracy': majority_accuracy,
                                  'Sensitivity': majority_sensitivity, 'Specificity': majority_specificity,
                                  'PPV': majority_ppv, 'NPV': majority_npv})

majority_fpr, majority_tpr, _ = roc_curve(y_test, y_proba, drop_intermediate=False)

# Create DataFrames for metrics
rf_train_metrics_df = pd.DataFrame(rf_train_metrics_list)
svm_train_metrics_df = pd.DataFrame(svm_train_metrics_list)
lr_train_metrics_df = pd.DataFrame(lr_train_metrics_list)

rf_metrics_df_soft = pd.DataFrame(rf_metrics_list)
svm_metrics_df_soft = pd.DataFrame(svm_metrics_list)
lr_metrics_df_soft = pd.DataFrame(lr_metrics_list)
majority_metrics_df_soft = pd.DataFrame(majority_metrics_list)

# Plot ROC curves
plt.figure(figsize=(8, 6))

plt.plot(rf_fpr, rf_tpr, label=f'Random Forest AUC = {rf_metrics_df_soft["AUC"].mean():.3f}')
plt.plot(svm_fpr, svm_tpr, label=f'SVM AUC = {svm_metrics_df_soft["AUC"].mean():.3f}')
plt.plot(lr_fpr, lr_tpr, label=f'Logistic Regression AUC = {lr_metrics_df_soft["AUC"].mean():.3f}')
plt.plot(majority_fpr, majority_tpr, label=f'Majority Vote AUC = {majority_metrics_df_soft["AUC"].mean():.3f}')

plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('')
plt.legend()
plt.show()

# Print average AUC values for each model in both training and test sets
print(f'The average AUC for Random Forest in the training set is: {rf_train_metrics_df["AUC"].mean()}')
print(f'The average AUC for SVM in the training set is: {svm_train_metrics_df["AUC"].mean()}')
print(f'The average AUC for Logistic Regression in the training set is: {lr_train_metrics_df["AUC"].mean()}')
print()
print(f'The AUC for Random Forest in the test set is: {rf_metrics_df_soft["AUC"].mean()}')
print(f'The AUC for SVM in the test set is: {svm_metrics_df_soft["AUC"].mean()}')
print(f'The AUC for Logistic Regression in the test set is: {lr_metrics_df_soft["AUC"].mean()}')
print(f'The AUC for the Majority Vote model in the test set is: {majority_metrics_df_soft["AUC"].mean()}')

# De Long test
a, delong_p_lr_mv = delong_test(y_test, lr_proba, y_proba)
b, delong_p_lr_rf = delong_test(y_test, lr_proba, rf_proba)
c, delong_p_lr_svm = delong_test(y_test, lr_proba, svm_proba)
d, delong_p_mv_rf = delong_test(y_test, y_proba, rf_proba)
e, delong_p_mv_svm = delong_test(y_test, y_proba, svm_proba)
f, delong_p_rf_svm = delong_test(y_test, rf_proba, svm_proba)

print(f"{delong_p_lr_mv}, {delong_p_lr_rf}, {delong_p_lr_svm}, {delong_p_mv_rf}, {delong_p_mv_svm}, {delong_p_rf_svm}")

In [ ]:
rf_train_metrics_df.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP3_train_results_rf.xlsx')
svm_train_metrics_df.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP3_train_results_svm.xlsx')
lr_train_metrics_df.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP3_train_results_lr.xlsx')

rf_metrics_df_soft.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP3_results_rf.xlsx')
svm_metrics_df_soft.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP3_results_svm.xlsx')
lr_metrics_df_soft.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP3_results_lr.xlsx')
majority_metrics_df_soft.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP3_results_majority.xlsx')

# STEP 4

In [ ]:
# Load data
file_path = '/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/df_step3_4_5.csv'
df_step4 = pd.read_csv(file_path)
df_test_step4 = df_step4.iloc[0:62,]
df_train_step4 =df_step4.iloc[62:,]

X_train = df_train_step4[['"original_glszm_ZoneEntropy"', '"original_shape_LeastAxisLength"']].values
y_train = df_train_step4['x'].values
X_test = df_test_step4[['"original_glszm_ZoneEntropy"', '"original_shape_LeastAxisLength"']].values
y_test = df_test_step4['x'].values

# Initialize models and preprocessing steps
scaler = StandardScaler()
oversampler = SMOTE(sampling_strategy='auto', random_state=42, k_neighbors=3)
randomforest = RandomForestClassifier()
svm = SVC(probability=True)
lr = LogisticRegression()

# Create pipelines
pipeline_rf = Pipeline(steps=[("oversampler", oversampler), ("model_rf", randomforest)])
pipeline_svm = Pipeline(steps=[("oversampler", oversampler), ("model_svm", svm)])
pipeline_lr = Pipeline(steps=[("oversampler", oversampler), ("model_lr", lr)])

# Define hyperparameter grids
rf_params = {'model_rf__n_estimators': [25, 50, 75, 100, 125, 150],
             'model_rf__min_samples_split': [2, 3],
             'model_rf__min_samples_leaf': [1, 2],
             'model_rf__max_features': [2]}

svm_params = {'model_svm__C': [0.5, 1, 1.5, 2, 2.5, 3],
              'model_svm__gamma': [0.5, 0.8, 1, 1.5],
              'model_svm__kernel': ['linear'],
              'model_svm__probability': [True]}

lr_params = {'model_lr__solver': ['lbfgs', 'newton-cg', 'liblinear'],
             'model_lr__penalty': ['l2'],
             'model_lr__C': [0.1, 0.5, 0.8, 1, 1.25, 2]}

# Initialize lists to store metrics
rf_train_metrics_list, svm_train_metrics_list, lr_train_metrics_list = [], [], []
rf_metrics_list, svm_metrics_list, lr_metrics_list, majority_metrics_list = [], [], [], []
    
# Scale features
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
    
# Resample training data
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train_scaled, y_train)

# Grid search for each model
rf_gs = GridSearchCV(pipeline_rf, rf_params, scoring='roc_auc')
svm_gs = GridSearchCV(pipeline_svm, svm_params, scoring='roc_auc')
lr_gs = GridSearchCV(pipeline_lr, lr_params, scoring='roc_auc')

rf_gs.fit(X_train_scaled, y_train)
svm_gs.fit(X_train_scaled, y_train)
lr_gs.fit(X_train_scaled, y_train)

# Training AUC and hyperparameters
rf_train_auc = rf_gs.best_score_
rf_train_metrics_list.append({'AUC': rf_train_auc, 'Hyperparameters': rf_gs.best_params_})

svm_train_auc = svm_gs.best_score_
svm_train_metrics_list.append({'AUC': svm_train_auc, 'Hyperparameters': svm_gs.best_params_})

lr_train_auc = lr_gs.best_score_
lr_train_metrics_list.append({'AUC': lr_train_auc, 'Hyperparameters': lr_gs.best_params_})

# Predictions and probabilities
rf_pred = rf_gs.predict(X_test_scaled)
svm_pred = svm_gs.predict(X_test_scaled)
lr_pred = lr_gs.predict(X_test_scaled)

rf_proba = rf_gs.predict_proba(X_test_scaled)[:, 1]
svm_proba = svm_gs.predict_proba(X_test_scaled)[:, 1]
lr_proba = lr_gs.predict_proba(X_test_scaled)[:, 1]

# Test set metrics for each model
rf_auc = roc_auc_score(y_test, rf_proba)
svm_auc = roc_auc_score(y_test, svm_proba)
lr_auc = roc_auc_score(y_test, lr_proba)

rf_metrics_list.append({'AUC': rf_auc, 'Accuracy': accuracy_score(y_test, rf_pred), 'Sensitivity': confusion_matrix(y_test, rf_pred)[1, 1] / (confusion_matrix(y_test, rf_pred)[1, 1] + confusion_matrix(y_test, rf_pred)[1, 0]),
                            'Specificity': confusion_matrix(y_test, rf_pred)[0, 0] / (confusion_matrix(y_test, rf_pred)[0, 0] + confusion_matrix(y_test, rf_pred)[0, 1]),
                            'PPV': confusion_matrix(y_test, rf_pred)[1, 1] / (confusion_matrix(y_test, rf_pred)[1, 1] + confusion_matrix(y_test, rf_pred)[0, 1]),
                            'NPV': confusion_matrix(y_test, rf_pred)[0, 0] / (confusion_matrix(y_test, rf_pred)[0, 0] + confusion_matrix(y_test, rf_pred)[1, 0]),
                            'Hyperparameters': rf_gs.best_params_})

svm_metrics_list.append({'AUC': svm_auc, 'Accuracy': accuracy_score(y_test, svm_pred), 'Sensitivity': confusion_matrix(y_test, svm_pred)[1, 1] / (confusion_matrix(y_test, svm_pred)[1, 1] + confusion_matrix(y_test, svm_pred)[1, 0]),
                             'Specificity': confusion_matrix(y_test, svm_pred)[0, 0] / (confusion_matrix(y_test, svm_pred)[0, 0] + confusion_matrix(y_test, svm_pred)[0, 1]),
                             'PPV': confusion_matrix(y_test, svm_pred)[1, 1] / (confusion_matrix(y_test, svm_pred)[1, 1] + confusion_matrix(y_test, svm_pred)[0, 1]),
                             'NPV': confusion_matrix(y_test, svm_pred)[0, 0] / (confusion_matrix(y_test, svm_pred)[0, 0] + confusion_matrix(y_test, svm_pred)[1, 0]),
                             'Hyperparameters': svm_gs.best_params_})

lr_metrics_list.append({'AUC': lr_auc, 'Accuracy': accuracy_score(y_test, lr_pred), 'Sensitivity': confusion_matrix(y_test, lr_pred)[1, 1] / (confusion_matrix(y_test, lr_pred)[1, 1] + confusion_matrix(y_test, lr_pred)[1, 0]),
                            'Specificity': confusion_matrix(y_test, lr_pred)[0, 0] / (confusion_matrix(y_test, lr_pred)[0, 0] + confusion_matrix(y_test, lr_pred)[0, 1]),
                            'PPV': confusion_matrix(y_test, lr_pred)[1, 1] / (confusion_matrix(y_test, lr_pred)[1, 1] + confusion_matrix(y_test, lr_pred)[0, 1]),
                            'NPV': confusion_matrix(y_test, lr_pred)[0, 0] / (confusion_matrix(y_test, lr_pred)[0, 0] + confusion_matrix(y_test, lr_pred)[1, 0]),
                            'Hyperparameters': lr_gs.best_params_})

# ROC curve for each model
rf_fpr, rf_tpr, _ = roc_curve(y_test, rf_proba, drop_intermediate=False)
svm_fpr, svm_tpr, _ = roc_curve(y_test, svm_proba, drop_intermediate=False)
lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_proba, drop_intermediate=False)


# Majority Vote model
clf = VotingClassifier(estimators=[
    ('rf', RandomForestClassifier(**{key.split('__')[1]: value for key, value in rf_gs.best_params_.items()})),
    ('svm', SVC(**{key.split('__')[1]: value for key, value in svm_gs.best_params_.items()})),
    ('lr', LogisticRegression(**{key.split('__')[1]: value for key, value in lr_gs.best_params_.items()}))
], voting='soft', flatten_transform=True)


clf.fit(X_train_resampled, y_train_resampled)
y_pred = clf.predict(X_test_scaled)
y_proba = clf.predict_proba(X_test_scaled)[:, 1]

# Majority Vote metrics
majority_auc = roc_auc_score(y_test, y_proba)
majority_cm = confusion_matrix(y_test, y_pred)
majority_accuracy = accuracy_score(y_test, y_pred)

majority_sensitivity = majority_cm[1, 1] / (majority_cm[1, 1] + majority_cm[1, 0])
majority_specificity = majority_cm[0, 0] / (majority_cm[0, 0] + majority_cm[0, 1])
majority_ppv = majority_cm[1, 1] / (majority_cm[1, 1] + majority_cm[0, 1])
majority_npv = majority_cm[0, 0] / (majority_cm[0, 0] + majority_cm[1, 0])

majority_metrics_list.append({'AUC': majority_auc, 'Accuracy': majority_accuracy,
                                  'Sensitivity': majority_sensitivity, 'Specificity': majority_specificity,
                                  'PPV': majority_ppv, 'NPV': majority_npv})

majority_fpr, majority_tpr, _ = roc_curve(y_test, y_proba, drop_intermediate=False)

# Create DataFrames for metrics
rf_train_metrics_df = pd.DataFrame(rf_train_metrics_list)
svm_train_metrics_df = pd.DataFrame(svm_train_metrics_list)
lr_train_metrics_df = pd.DataFrame(lr_train_metrics_list)

rf_metrics_df_soft = pd.DataFrame(rf_metrics_list)
svm_metrics_df_soft = pd.DataFrame(svm_metrics_list)
lr_metrics_df_soft = pd.DataFrame(lr_metrics_list)
majority_metrics_df_soft = pd.DataFrame(majority_metrics_list)

# Plot ROC curves
plt.figure(figsize=(8, 6))

plt.plot(rf_fpr, rf_tpr, label=f'Random Forest AUC = {rf_metrics_df_soft["AUC"].mean():.3f}')
plt.plot(svm_fpr, svm_tpr, label=f'SVM AUC = {svm_metrics_df_soft["AUC"].mean():.3f}')
plt.plot(lr_fpr, lr_tpr, label=f'Logistic Regression AUC = {lr_metrics_df_soft["AUC"].mean():.3f}')
plt.plot(majority_fpr, majority_tpr, label=f'Majority Vote AUC = {majority_metrics_df_soft["AUC"].mean():.3f}')

plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('')
plt.legend()
plt.show()

# Print average AUC values for each model in both training and test sets
print(f'The average AUC for Random Forest in the training set is: {rf_train_metrics_df["AUC"].mean()}')
print(f'The average AUC for SVM in the training set is: {svm_train_metrics_df["AUC"].mean()}')
print(f'The average AUC for Logistic Regression in the training set is: {lr_train_metrics_df["AUC"].mean()}')
print()
print(f'The AUC for Random Forest in the test set is: {rf_metrics_df_soft["AUC"].mean()}')
print(f'The AUC for SVM in the test set is: {svm_metrics_df_soft["AUC"].mean()}')
print(f'The AUC for Logistic Regression in the test set is: {lr_metrics_df_soft["AUC"].mean()}')
print(f'The AUC for the Majority Vote model in the test set is: {majority_metrics_df_soft["AUC"].mean()}')

# De Long test
a, delong_p_lr_mv = delong_test(y_test, lr_proba, y_proba)
b, delong_p_lr_rf = delong_test(y_test, lr_proba, rf_proba)
c, delong_p_lr_svm = delong_test(y_test, lr_proba, svm_proba)
d, delong_p_mv_rf = delong_test(y_test, y_proba, rf_proba)
e, delong_p_mv_svm = delong_test(y_test, y_proba, svm_proba)
f, delong_p_rf_svm = delong_test(y_test, rf_proba, svm_proba)

print(f"{delong_p_lr_mv}, {delong_p_lr_rf}, {delong_p_lr_svm}, {delong_p_mv_rf}, {delong_p_mv_svm}, {delong_p_rf_svm}")

In [ ]:
rf_train_metrics_df.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP4_train_results_rf.xlsx')
svm_train_metrics_df.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP4_train_results_svm.xlsx')
lr_train_metrics_df.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP4_train_results_lr.xlsx')

rf_metrics_df_soft.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP4_results_rf.xlsx')
svm_metrics_df_soft.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP4_results_svm.xlsx')
lr_metrics_df_soft.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP4_results_lr.xlsx')
majority_metrics_df_soft.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP4_results_majority.xlsx')

# STEP 5

In [ ]:
# Load data
file_path = '/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/df_step3_4_5.csv'
df_step5 = pd.read_csv(file_path)

# Prepare features and target
X5 = df_step5[['"original_glszm_ZoneEntropy"', '"original_shape_LeastAxisLength"']].values
y5 = df_step5['x'].values

# Initialize models and preprocessing steps
scaler = StandardScaler()
oversampler = SMOTE(sampling_strategy='auto', random_state=42, k_neighbors=3)
randomforest = RandomForestClassifier()
svm = SVC(probability=True)
lr = LogisticRegression()

# Create pipelines
pipeline_rf = Pipeline(steps=[("oversampler", oversampler), ("model_rf", randomforest)])
pipeline_svm = Pipeline(steps=[("oversampler", oversampler), ("model_svm", svm)])
pipeline_lr = Pipeline(steps=[("oversampler", oversampler), ("model_lr", lr)])

# Define hyperparameter grids
rf_params = {'model_rf__n_estimators': [25, 50, 75, 100, 125, 150],
             'model_rf__min_samples_split': [2, 3],
             'model_rf__min_samples_leaf': [1, 2],
             'model_rf__max_features': [2]}

svm_params = {'model_svm__C': [0.5, 1, 1.5, 2, 2.5, 3],
              'model_svm__gamma': [0.5, 0.8, 1, 1.5],
              'model_svm__kernel': ['linear'],
              'model_svm__probability': [True]}

lr_params = {'model_lr__solver': ['lbfgs', 'newton-cg', 'liblinear'],
             'model_lr__penalty': ['l2'],
             'model_lr__C': [0.1, 0.5, 0.8, 1, 1.25, 2]}

# Initialize lists to store metrics
num_zeros_list, num_ones_list = [], []
rf_train_metrics_list, svm_train_metrics_list, lr_train_metrics_list = [], [], []
rf_metrics_list, svm_metrics_list, lr_metrics_list, majority_metrics_list = [], [], [], []
rf_all_fpr, rf_all_tpr, svm_all_fpr, svm_all_tpr, lr_all_fpr, lr_all_tpr, majority_all_fpr, majority_all_tpr = [], [], [], [], [], [], [], []

# Loop over iterations
for i in tqdm(range(100)):
    time.sleep(0.1)
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X5, y5, test_size=0.3, stratify=y5, random_state=i)
    
    # Scale features
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Resample training data
    X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train_scaled, y_train)

    # Grid search for each model
    rf_gs = GridSearchCV(pipeline_rf, rf_params, scoring='roc_auc')
    svm_gs = GridSearchCV(pipeline_svm, svm_params, scoring='roc_auc')
    lr_gs = GridSearchCV(pipeline_lr, lr_params, scoring='roc_auc')

    rf_gs.fit(X_train_scaled, y_train)
    svm_gs.fit(X_train_scaled, y_train)
    lr_gs.fit(X_train_scaled, y_train)

    # Training AUC and hyperparameters
    rf_train_auc = rf_gs.best_score_
    rf_train_metrics_list.append({'AUC': rf_train_auc, 'Hyperparameters': rf_gs.best_params_})

    svm_train_auc = svm_gs.best_score_
    svm_train_metrics_list.append({'AUC': svm_train_auc, 'Hyperparameters': svm_gs.best_params_})

    lr_train_auc = lr_gs.best_score_
    lr_train_metrics_list.append({'AUC': lr_train_auc, 'Hyperparameters': lr_gs.best_params_})

    # Predictions and probabilities
    rf_pred = rf_gs.predict(X_test_scaled)
    svm_pred = svm_gs.predict(X_test_scaled)
    lr_pred = lr_gs.predict(X_test_scaled)

    rf_proba = rf_gs.predict_proba(X_test_scaled)[:, 1]
    svm_proba = svm_gs.predict_proba(X_test_scaled)[:, 1]
    lr_proba = lr_gs.predict_proba(X_test_scaled)[:, 1]

    # Test set metrics for each model
    rf_auc = roc_auc_score(y_test, rf_proba)
    svm_auc = roc_auc_score(y_test, svm_proba)
    lr_auc = roc_auc_score(y_test, lr_proba)

    rf_metrics_list.append({'AUC': rf_auc, 'Accuracy': accuracy_score(y_test, rf_pred), 'Sensitivity': confusion_matrix(y_test, rf_pred)[1, 1] / (confusion_matrix(y_test, rf_pred)[1, 1] + confusion_matrix(y_test, rf_pred)[1, 0]),
                            'Specificity': confusion_matrix(y_test, rf_pred)[0, 0] / (confusion_matrix(y_test, rf_pred)[0, 0] + confusion_matrix(y_test, rf_pred)[0, 1]),
                            'PPV': confusion_matrix(y_test, rf_pred)[1, 1] / (confusion_matrix(y_test, rf_pred)[1, 1] + confusion_matrix(y_test, rf_pred)[0, 1]),
                            'NPV': confusion_matrix(y_test, rf_pred)[0, 0] / (confusion_matrix(y_test, rf_pred)[0, 0] + confusion_matrix(y_test, rf_pred)[1, 0]),
                            'Hyperparameters': rf_gs.best_params_})

    svm_metrics_list.append({'AUC': svm_auc, 'Accuracy': accuracy_score(y_test, svm_pred), 'Sensitivity': confusion_matrix(y_test, svm_pred)[1, 1] / (confusion_matrix(y_test, svm_pred)[1, 1] + confusion_matrix(y_test, svm_pred)[1, 0]),
                             'Specificity': confusion_matrix(y_test, svm_pred)[0, 0] / (confusion_matrix(y_test, svm_pred)[0, 0] + confusion_matrix(y_test, svm_pred)[0, 1]),
                             'PPV': confusion_matrix(y_test, svm_pred)[1, 1] / (confusion_matrix(y_test, svm_pred)[1, 1] + confusion_matrix(y_test, svm_pred)[0, 1]),
                             'NPV': confusion_matrix(y_test, svm_pred)[0, 0] / (confusion_matrix(y_test, svm_pred)[0, 0] + confusion_matrix(y_test, svm_pred)[1, 0]),
                             'Hyperparameters': svm_gs.best_params_})

    lr_metrics_list.append({'AUC': lr_auc, 'Accuracy': accuracy_score(y_test, lr_pred), 'Sensitivity': confusion_matrix(y_test, lr_pred)[1, 1] / (confusion_matrix(y_test, lr_pred)[1, 1] + confusion_matrix(y_test, lr_pred)[1, 0]),
                            'Specificity': confusion_matrix(y_test, lr_pred)[0, 0] / (confusion_matrix(y_test, lr_pred)[0, 0] + confusion_matrix(y_test, lr_pred)[0, 1]),
                            'PPV': confusion_matrix(y_test, lr_pred)[1, 1] / (confusion_matrix(y_test, lr_pred)[1, 1] + confusion_matrix(y_test, lr_pred)[0, 1]),
                            'NPV': confusion_matrix(y_test, lr_pred)[0, 0] / (confusion_matrix(y_test, lr_pred)[0, 0] + confusion_matrix(y_test, lr_pred)[1, 0]),
                            'Hyperparameters': lr_gs.best_params_})

    # ROC curve for each model
    rf_fpr, rf_tpr, _ = roc_curve(y_test, rf_proba, drop_intermediate=False)
    interp_rf_tpr = np.interp(np.linspace(0, 1, 100), rf_fpr, rf_tpr)
    rf_all_fpr.append(np.linspace(0, 1, 100))
    rf_all_tpr.append(interp_rf_tpr)

    svm_fpr, svm_tpr, _ = roc_curve(y_test, svm_proba, drop_intermediate=False)
    interp_svm_tpr = np.interp(np.linspace(0, 1, 100), svm_fpr, svm_tpr)
    svm_all_fpr.append(np.linspace(0, 1, 100))
    svm_all_tpr.append(interp_svm_tpr)

    lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_proba, drop_intermediate=False)
    interp_lr_tpr = np.interp(np.linspace(0, 1, 100), lr_fpr, lr_tpr)
    lr_all_fpr.append(np.linspace(0, 1, 100))
    lr_all_tpr.append(interp_lr_tpr)

    # Majority Vote model
    clf = VotingClassifier(estimators=[
    ('rf', RandomForestClassifier(**{key.split('__')[1]: value for key, value in rf_gs.best_params_.items()})),
    ('svm', SVC(**{key.split('__')[1]: value for key, value in svm_gs.best_params_.items()})),
    ('lr', LogisticRegression(**{key.split('__')[1]: value for key, value in lr_gs.best_params_.items()}))
    ], voting='soft', flatten_transform=True)

    clf.fit(X_train_resampled, y_train_resampled)
    y_pred = clf.predict(X_test_scaled)
    y_proba = clf.predict_proba(X_test_scaled)[:, 1]

    # Majority Vote metrics
    majority_auc = roc_auc_score(y_test, y_proba)
    majority_cm = confusion_matrix(y_test, y_pred)
    majority_accuracy = accuracy_score(y_test, y_pred)

    majority_sensitivity = majority_cm[1, 1] / (majority_cm[1, 1] + majority_cm[1, 0])
    majority_specificity = majority_cm[0, 0] / (majority_cm[0, 0] + majority_cm[0, 1])
    majority_ppv = majority_cm[1, 1] / (majority_cm[1, 1] + majority_cm[0, 1])
    majority_npv = majority_cm[0, 0] / (majority_cm[0, 0] + majority_cm[1, 0])

    majority_metrics_list.append({'AUC': majority_auc, 'Accuracy': majority_accuracy,
                                  'Sensitivity': majority_sensitivity, 'Specificity': majority_specificity,
                                  'PPV': majority_ppv, 'NPV': majority_npv})

    majority_fpr, majority_tpr, _ = roc_curve(y_test, y_proba, drop_intermediate=False)
    interp_majority_tpr = np.interp(np.linspace(0, 1, 100), majority_fpr, majority_tpr)
    majority_all_fpr.append(np.linspace(0, 1, 100))
    majority_all_tpr.append(interp_majority_tpr)

    # Store class distribution in each iteration
    num_zeros_list.append(np.sum(y_test == 0))
    num_ones_list.append(np.sum(y_test == 1))

# Create DataFrames for metrics
rf_train_metrics_df = pd.DataFrame(rf_train_metrics_list)
svm_train_metrics_df = pd.DataFrame(svm_train_metrics_list)
lr_train_metrics_df = pd.DataFrame(lr_train_metrics_list)

rf_metrics_df_soft = pd.DataFrame(rf_metrics_list)
svm_metrics_df_soft = pd.DataFrame(svm_metrics_list)
lr_metrics_df_soft = pd.DataFrame(lr_metrics_list)
majority_metrics_df_soft = pd.DataFrame(majority_metrics_list)

# Plot ROC curves
plt.figure(figsize=(8, 6))

rf_mean_fpr = np.mean(rf_all_fpr, axis=0)
rf_mean_tpr = np.mean(rf_all_tpr, axis=0)
plt.plot(rf_mean_fpr, rf_mean_tpr, label=f'Random Forest AUC = {np.mean(rf_metrics_df_soft["AUC"]):.3f}')


svm_mean_fpr = np.mean(svm_all_fpr, axis=0)
svm_mean_tpr = np.mean(svm_all_tpr, axis=0)
plt.plot(svm_mean_fpr, svm_mean_tpr, label=f'SVM AUC = {np.mean(svm_metrics_df_soft["AUC"]):.3f}')

lr_mean_fpr = np.mean(lr_all_fpr, axis=0)
lr_mean_tpr = np.mean(lr_all_tpr, axis=0)
lr_mean_fpr = np.concatenate([[0], lr_mean_fpr])
lr_mean_tpr = np.concatenate([[0], lr_mean_tpr])
plt.plot(lr_mean_fpr, lr_mean_tpr, label=f'Logistic Regression AUC = {np.mean(lr_metrics_df_soft["AUC"]):.3f}')

majority_mean_fpr = np.mean(majority_all_fpr, axis=0)
majority_mean_tpr = np.mean(majority_all_tpr, axis=0)
majority_mean_fpr = np.concatenate([[0], majority_mean_fpr])
majority_mean_tpr = np.concatenate([[0], majority_mean_tpr])
plt.plot(majority_mean_fpr, majority_mean_tpr, label=f'Majority Vote AUC = {np.mean(majority_metrics_df_soft["AUC"]):.3f}')

plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('')
plt.legend()
plt.show()

# Create a DataFrame for class distribution
sanity_check_df = pd.DataFrame({'Num_Zeros': num_zeros_list, 'Num_Ones': num_ones_list})

# Print average AUC values for each model in both training and test sets
print(f'The average AUC for Random Forest in the training set is: {rf_train_metrics_df["AUC"].mean()}')
print(f'The average AUC for SVM in the training set is: {svm_train_metrics_df["AUC"].mean()}')
print(f'The average AUC for Logistic Regression in the training set is: {lr_train_metrics_df["AUC"].mean()}')
print()
print(f'The average AUC for Random Forest in the test set is: {rf_metrics_df_soft["AUC"].mean()}')
print(f'The average AUC for SVM in the test set is: {svm_metrics_df_soft["AUC"].mean()}')
print(f'The average AUC for Logistic Regression in the test set is: {lr_metrics_df_soft["AUC"].mean()}')
print(f'The average AUC for the Majority Vote model in the test set is: {majority_metrics_df_soft["AUC"].mean()}')

In [ ]:
rf_train_metrics_df.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP5_train_results_rf.xlsx')
svm_train_metrics_df.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP5_train_results_svm.xlsx')
lr_train_metrics_df.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP5_train_results_lr.xlsx')

rf_metrics_df_soft.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP5_results_rf.xlsx')
svm_metrics_df_soft.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP5_results_svm.xlsx')
lr_metrics_df_soft.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP5_results_lr.xlsx')
majority_metrics_df_soft.to_excel('/beegfs/scratch/ric.medicinanucleare/ghezzo.samuele/OSR_STN_CLUSTER/STEP5_results_majority.xlsx')